In [86]:
#Bibliotecas Utilizadas
import IPython as IP
import CoolProp
import CoolProp.CoolProp as CP #Propriedades termodinâmicas
import CoolProp.Plots as CPP #Plotando gráficos
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Gráfico P-H de um Sistema de Resfriamento Típico

![](Pressure-enthalpy-diagram-for-a-standard-vapor-compression-cycle-with-four-thermodynamic.png)

## Modelagem - Cálculo de Eficiências de um Compressor (1 Caso)

In [74]:
# Funções Auxiliares

# Convertendo de graus Célcius para graus Kelvin
def fromCtoK (x):
    return x + 273.15;

In [75]:
# Setando Líquido Refrigerante
coolant = 'R410A'

# Condições iniciais de operação
T_cond = 54.4 # Temperatura de Condensação (ºC)
T_evap = -23.3 # Temperatura de Evaporação (ºC)
T_sup = 32 # Temperatura de superaquecimento (ºC)
T_sub = 32 # Temperatura de subesfriamento (ºC)
P_cond = CP.PropsSI('P','Q',1,'T',fromCtoK(T_cond),coolant) #Pressão de Saturação na região de condensação (Pa)
P_evap = CP.PropsSI('P','Q',1,'T',fromCtoK(T_evap),coolant) #Pressão de Saturação na região de evaporação (Pa)
Q_evap = 88 # Carga Térmica do Evaporador (W)
ROT = 60 # Rotação do pistão do compressor (RPS)
ROT_max = 120 # Rotação máxima do pistão do compressor (RPS)

#Variáreis de Controle
RP = P_cond/P_evap # Razão de Pressões
Sigma = ROT/ROT_max # Razão entre a rotação de operação e a rotação máxima

print('Satured Pressure of R410A at T_cond :', P_cond,'kPa')
print('Satured Pressure of R410A at T_evap :', P_evap, 'KPa')

Satured Pressure of R410A at T_cond : 3385602.299623575 kPa
Satured Pressure of R410A at T_evap : 352023.5637278192 KPa


In [76]:
# Evaporador
h_1 = CP.PropsSI('H','T',fromCtoK(T_sup),'P',P_evap,coolant) # Entalpia no ponto 1
h_3 = CP.PropsSI('H','T',fromCtoK(T_sub),'P',P_cond,coolant) # Entalpia no Ponto 3, que equivale ao ponto 4
m_r = Q_evap/(h_1 - h_3) # Fluxo mássico no evaporador

In [77]:
# Compressor
V_cil = 2.83e-06 # Volume total dos cilindros (mˆ3)
v_1 = 1/CP.PropsSI('D','T',fromCtoK(T_sup),'P',P_evap,coolant) # Volume específico na entrada do compressor (mˆ3/Kg)
eta_v = (m_r*v_1)/(ROT*V_cil)# Eficiência Volumétrica do Compressor

W_real = 73 # Trabalho real do compressor (W)
s_1 = CP.PropsSI('S','T',fromCtoK(T_sup),'P',P_evap,coolant) # Entropia na entrada do compressor
h_2s = CP.PropsSI('H','S',s_1,'P',P_cond,coolant) # Entropia na saída do compressor (Processo Isentrópico)
W_s = m_r*(h_2s-h_1) # Trabalho ideal do compressor (W)
eta_g = W_s/W_real # Eficiência Global do Compressor

In [78]:
# Eficiências do compressor 
print('Eficiência Volumétrica:',eta_v)
print('Eficiência Global:',eta_g)

Eficiência Volumétrica: 0.2337047454626563
Eficiência Global: 0.4913917871062722


## Modelagem - Cálculo de Eficiências de um Compressor (Múltiplos Casos)

### Variação da Eficiência Volumétrica com a Temperatura do Evaporador

In [79]:
def Calc_EficVolumetrica_TempEvap (T_evap, T_cond=54.4, T_sup=32, T_sub=32,Q_evap=88,ROT=60,v_CIL=2.83e-06,W_real=73,coolant='R410A'):
    P_evap = CP.PropsSI('P','Q',1,'T',fromCtoK(T_evap),coolant)
    h_1 = CP.PropsSI('H','T',fromCtoK(T_sup),'P',P_evap,coolant)
    h_3 = CP.PropsSI('H','T',fromCtoK(T_sub),'P',P_cond,coolant)
    m_r = Q_evap/(h_1 - h_3)
    v_1 = 1/CP.PropsSI('D','T',fromCtoK(T_sup),'P',P_evap,coolant)
    return (m_r*v_1)/(ROT*V_cil)


In [81]:
T_values = np.linspace(0,20,10)

In [83]:
y_Values = []
for T in T_values:
    y_Values.append(Calc_EficVolumetrica_TempEvap(T))

In [93]:
# Plotando o gráfico de Eficiência Volumétria por Temperatura do Evaporador
fig = go.Figure()
fig.add_trace(go.Scatter(x=T_values, y=y_Values,
                    line_shape='spline', name='Eficiencia Volumetrica'))
fig.update_layout(title='Eficiência Volumétria do Compressor X Temperatura do Evaporador',
                   xaxis_title='Temperatura do Evaporador (ºC)',
                   yaxis_title='Eficiência Volumétrica do Compressor')
fig.show()

### Variação da Eficiência Global com a Temperatura do Evaporador

In [94]:
def Calc_EficGlobal_TempEvap (T_evap, T_cond=54.4, T_sup=32, T_sub=32,Q_evap=88,ROT=60,v_CIL=2.83e-06,W_real=73,coolant='R410A'):
    P_evap = CP.PropsSI('P','Q',1,'T',fromCtoK(T_evap),coolant)
    h_1 = CP.PropsSI('H','T',fromCtoK(T_sup),'P',P_evap,coolant)
    h_3 = CP.PropsSI('H','T',fromCtoK(T_sub),'P',P_cond,coolant)
    m_r = Q_evap/(h_1 - h_3)
    s_1 = CP.PropsSI('S','T',fromCtoK(T_sup),'P',P_evap,coolant)
    h_2s = CP.PropsSI('H','S',s_1,'P',P_cond,coolant)
    W_s = m_r*(h_2s-h_1)
    return W_s/W_real

In [95]:
z_Values = []
for T in T_values:
    z_Values.append(Calc_EficGlobal_TempEvap(T))

In [96]:
# Plotando o gráfico de Eficiência Volumétria por Temperatura do Evaporador
fig.add_trace(go.Scatter(x=T_values, y=z_Values,
                    line_shape='spline', name='Eficiencia Global'))
fig.update_layout(title='Eficiências do Compressor X Temperatura do Evaporador',
                   xaxis_title='Temperatura do Evaporador (ºC)',
                   yaxis_title='Eficiência Global do Compressor')
fig.show()